# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
spaceship.shape

(8693, 14)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
clean_spaceship = spaceship.dropna()

In [5]:
clean_spaceship['Cabin'] = clean_spaceship['Cabin'].apply(lambda x: x.split('/')[0])

/var/folders/ns/cnr3tjln5ns_6pvq3ylpgl9m0000gn/T/ipykernel_67346/288168877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_spaceship['Cabin'] = clean_spaceship['Cabin'].apply(lambda x: x.split('/')[0])


In [6]:
clean_spaceship.drop(['PassengerId','Name'], axis=1, inplace=True)

/var/folders/ns/cnr3tjln5ns_6pvq3ylpgl9m0000gn/T/ipykernel_67346/2652508835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_spaceship.drop(['PassengerId','Name'], axis=1, inplace=True)


In [7]:
# dummification
cat_col = clean_spaceship.select_dtypes(include=['object'])
df_dummies = pd.get_dummies(cat_col)
num_col = clean_spaceship.select_dtypes(exclude=['object'])

# Combine the non-categorical columns with the dummies
dummy_spaceship = pd.concat([num_col, df_dummies], axis=1)

dummy_spaceship.head(2)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,True,False,False,...,False,False,True,False,False,False,False,True,True,False


In [8]:
features = dummy_spaceship.drop(columns = ['Transported'])
target = dummy_spaceship['Transported']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [27]:
# imported 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [11]:
# initialize scaler
normalizer = MinMaxScaler()
# train the scaler
normalizer.fit(X_train)

MinMaxScaler()

In [13]:
# transform
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [14]:
# turining the arrays into pandas dataframes
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [28]:
forest = RandomForestRegressor(n_estimators=100, max_depth=20)

In [29]:
forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=20)

In [30]:
pred = forest.predict(X_test_norm)

- Evaluate your model

In [32]:
print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"RMSE, {mean_squared_error(pred, y_test, squared=False): .2f}")
print(f"R2 score, {forest.score(X_test_norm, y_test): .2f}")

MAE,  0.27
RMSE,  0.38
R2 score,  0.42


/opt/anaconda3/envs/notebook/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [19]:
from sklearn.model_selection import GridSearchCV

In [40]:
grid = {"max_leaf_nodes": [25, 50, 75, 100],
        "max_depth":[10,30,50]}

- Run Grid Search

In [41]:
model = GridSearchCV(estimator = forest, param_grid = grid, cv=5, verbose=10)

In [42]:
model.fit(X_train_norm, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=10, max_leaf_nodes=100............................
[CV 1/5; 1/9] END max_depth=10, max_leaf_nodes=100;, score=0.477 total time=   1.2s
[CV 2/5; 1/9] START max_depth=10, max_leaf_nodes=100............................
[CV 2/5; 1/9] END max_depth=10, max_leaf_nodes=100;, score=0.479 total time=   1.3s
[CV 3/5; 1/9] START max_depth=10, max_leaf_nodes=100............................
[CV 3/5; 1/9] END max_depth=10, max_leaf_nodes=100;, score=0.440 total time=   1.2s
[CV 4/5; 1/9] START max_depth=10, max_leaf_nodes=100............................
[CV 4/5; 1/9] END max_depth=10, max_leaf_nodes=100;, score=0.436 total time=   1.1s
[CV 5/5; 1/9] START max_depth=10, max_leaf_nodes=100............................
[CV 5/5; 1/9] END max_depth=10, max_leaf_nodes=100;, score=0.453 total time=   1.1s
[CV 1/5; 2/9] START max_depth=10, max_leaf_nodes=150............................
[CV 1/5; 2/9] END max_depth=10, ma

GridSearchCV(cv=5, estimator=RandomForestRegressor(max_depth=20),
             param_grid={'max_depth': [10, 30, 50],
                         'max_leaf_nodes': [100, 150, 250]},
             verbose=10)

- Evaluate your model

In [47]:
# checking for the best combination of tested hyperparameters 
model.best_params_

{'max_depth': 30, 'max_leaf_nodes': 100}

In [48]:
# retrieve the best model with the best parameters
best_model = model.best_estimator_

In [49]:
pred_bm = best_model.predict(X_test_norm)

In [50]:
print(f"MAE: {mean_absolute_error(pred, y_test): .2f}")
print(f"RMSE: {mean_squared_error(pred, y_test, squared=False): .2f}")
print(f"R2 score:  {best_model.score(X_test_norm, y_test): .2f}")

MAE:  0.27
RMSE:  0.38
R2 score:   0.43


/opt/anaconda3/envs/notebook/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
